In [ ]:
import requests
import os
import json
import urllib.request
import pandas as pd
from bs4 import BeautifulSoup
Main_Languages = ['Haskell']

In [ ]:
problems = pd.read_csv('problems.csv',header=0)

In [ ]:
#f"/page/{Page_number}?order=BY_PROGRAM_LENGTH_ASC"

In [ ]:
def link_generator(problems):
  Links=[]
  for i, problem in problems.iterrows():
    contest_id = problem['contest_id']
    index = problem['index']
    Link_to_status = f"https://codeforces.com/problemset/status/{contest_id}/problem/{index}"
    Link_to_problem = f"https://codeforces.com/problemset/problem/{contest_id}/{index}"
    Links.append((Link_to_problem,Link_to_status))
  return Links
Links = link_generator(problems)

In [ ]:
Links[13]

('https://codeforces.com/problemset/problem/1370/A',
 'https://codeforces.com/problemset/status/1370/problem/A')

In [ ]:
len(Links)

6167

In [ ]:
def extract_links_ids_from_status_page(soup):
  """
  https://codeforces.com/problemset/status/1370/problem/D/page/1?order=BY_PROGRAM_LENGTH_ASC
  status page of one problem contain multible solutions
  I exctracted the solutions link and submissions id  that has language in MainLanguage list
  """
  table_of_submissions = soup.find('table', class_='status-frame-datatable')

  i=0
  Links_Ids = []

  #Access submission table

  for row in table_of_submissions.find_all('tr'):
    #Access cells in submission table
    cells = row.find_all('td')

    if len(cells) == 0:
      continue
    #Access Lang cell
    if len(cells) >= 5:
      language = cells[4].text.strip()
      if language not in Main_Languages:
        continue

    #Access "#" Cell
      first_cell = cells[0]
      a_tag = first_cell.find('a', class_='view-source')

      if a_tag:
        Link_to_source_solution = a_tag['href']
        submission_id = a_tag['submissionid']
        Links_Ids.append(("https://codeforces.com" + Link_to_source_solution , submission_id))
      else:
        print("Error2 - There is no a_Tag")
    else:
      print("Error1 - There is no language value in this row")
    i+=1
  return Links_Ids

In [ ]:

extracted_data=[]
URL = Links[13][1]
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
def extract_submissions_links_from_all_status(soup, link):
  pagination_div = soup.find_all('div', class_="pagination")[-1]
  last_page_number = -1
  if pagination_div:
    ul_element = pagination_div.find('ul')
    if ul_element:
      last_page_number = ul_element.find_all('li')[-2].text.strip()
    else:
      print("Error - no <ul>")
      print(link)
  else:
    print("Error - no <li>")

  #print(last_page_number)

  if last_page_number == -1:
    last_page_number = 10

  for page_number in range(1,int(last_page_number)+1):
    URL_pages = URL +  f"/page/{page_number}?order=BY_PROGRAM_LENGTH_ASC"
    other_page = requests.get(URL_pages)
    other_soup = BeautifulSoup(other_page.content, "html.parser")
    extracted_data.extend(extract_links_ids_from_status_page(other_soup))
    #print(len(extracted_data))
  return extracted_data


In [ ]:
Link1= "https://codeforces.com/problemset/submission/1370/84476054"
page = requests.get(Link1)
soup = BeautifulSoup(page.content, 'html.parser')
def extract_code_from_submission(soup):
  code = soup.find('pre', id='program-source-text').text.strip()
  return code
code = extract_code_from_submission(soup)

In [ ]:
print(code)

main = interact $ unlines . map (show . (flip div 2) . read) . tail . words


In [ ]:
from numpy.core.multiarray import empty
from bs4.element import NavigableString
def extract_text_mathlb_from_problem(position):
  data = ""
  math_pos = []
  try:
    if len(position.contents) > 1:
      for child in position.children:
          X , math_pos1 = extract_text_mathlb_from_problem(child)
          if child.name == "div" or child.name == "p":
              data += "\n"
              math_pos.extend([0,0])
          else:
              data += " "
              math_pos.extend([0])
          data += X
          math_pos.extend(math_pos1)
    else:
      if position.name == 'span' and position.get('class') == ['MathJax']:
          data = position.get('data-mathml')

          math_pos.extend([1 for i in range(len(data))])
      else:
          data = position.get_text(strip=False)
          math_pos.extend([0 for i in range(len(data))])
  except:
    if str(position).strip():
      data = str(position).strip()
      math_pos.extend([0 for i in range(len(data))])
  return data , math_pos

In [ ]:
Link1= "https://codeforces.com/contest/1370/problem/F1"
page = requests.get(Link1)
soup = BeautifulSoup(page.content, 'html.parser')
problem_statement = soup.find(class_='problem-statement')
Data , pos = extract_text_mathlb_from_problem(problem_statement)

In [ ]:
Data

'\n\nF1. The Hidden Pair (Easy Version)\n\ntime limit per test 2 seconds\n\nmemory limit per test 256 megabytes\n\ninput standard input\n\noutput standard output\n\nNote that the only difference between the easy and hard version is the constraint on the number of queries. You can make hacks only if all versions of the problem are solved.\nThis is an interactive problem.\nYou are given a tree consisting of $$$n$$$ nodes numbered with integers from $$$1$$$ to $$$n$$$. Ayush and Ashish chose two secret distinct nodes in the tree. You need to find out both the nodes. You can make the following query:     Provide a list of nodes and you will receive a node from that list whose sum of distances to both the hidden nodes is minimal (if there are multiple such nodes in the list, you will receive any one of them). You will also get the sum of distances of that node to the hidden nodes. \nRecall that a tree is a connected graph without cycles. The distance between two nodes is defined as the numb

In [ ]:
len(Links)

6167

In [ ]:
import concurrent.futures
import requests
from bs4 import BeautifulSoup

DataSet = []

def make_json(Links):
    j = 0

    def process_link(link):
        problem_link = link[0]
        status_link = link[1]
        try:
          page = requests.get(problem_link)
          soup = BeautifulSoup(page.content, 'html.parser')
          problem_statement = soup.find(class_='problem-statement')
          problem_data, pos = extract_text_mathlb_from_problem(problem_statement)

          try:
            page = requests.get(status_link)
            soup = BeautifulSoup(page.content, "html.parser")
            submissions_links = extract_submissions_links_from_all_status(soup , status_link)
            print(problem_link,len(submissions_links))
            for submission in submissions_links:
              try:
                page = requests.get(submission[0])
                soup = BeautifulSoup(page.content, 'html.parser')
                code = extract_code_from_submission(soup)

                DATA_FORM = {
                    'problem_statement': problem_data,
                    'solution': code,
                    'solution_link': submission[0],
                    'solution_id': submission[0],
                    'problem_link': problem_link
                }
                DataSet.append(DATA_FORM)
                nonlocal j
                j += 1
                print(j)
              except:
                print(f"could not access to the submission {submission[0]}")
          except:
            print(f"could not access to the status_link {status_link}")
        except:
          print(f"could not access to the problem link {problem_link}")

    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        executor.map(process_link, Links)

make_json(Links)

https://codeforces.com/problemset/problem/1373/G 0
https://codeforces.com/problemset/problem/1373/F 2
1
2
3
https://codeforces.com/problemset/problem/1373/E 5
4
5
6
7
8
https://codeforces.com/problemset/problem/1373/D 21
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
https://codeforces.com/problemset/problem/1373/C 25
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
https://codeforces.com/problemset/problem/1370/F2 29
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
could not access to the submission https://codeforces.com/problemset/submission/1370/117005752
could not access to the submission https://codeforces.com/problemset/submission/1370/119546710
could not access to the submission https://codeforces.com/problemset/submission/1370/84437157
could not access to the submission https://codeforces.com/problemset/submission/1370/84700462
could not access to the submission https://codeforces.com/problemset/submission/1370

KeyboardInterrupt: ignored

In [ ]:
from google.colab import files
import json
# Convert the data to JSON string
json_data = json.dumps(DataSet)

# Save the JSON string to a file
file_path = "data.json"
with open(file_path, "w") as file:
    file.write(json_data)

# Download the JSON file
files.download(file_path)

In [ ]:
!pip install -U -q PyDrive


In [ ]:
!zip archive.zip /content/data.json

In [ ]:
!cp /content/archive.zip /content/drive/MyDrive/codeforces.zip